In [1]:
!pip install galois
!pip install scipy

/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found
/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found


In [1]:
import galois
import numpy as np
import time
import os
from scipy import stats
import csv

In [45]:
#definisco una matrice k x (n-k)
#k = 126 #le righe
#n_k = 126 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 visto da pdf myLESS_CF pagina 38 di 45

In [46]:
field = galois.GF(127)


In [55]:
max_column = 12

In [48]:
#FUNCTION FOR CF 3
def has_duplicate_rows(matrix):
    _, counts = np.unique(matrix, axis=0, return_counts=True)
    return np.any(counts > 1)

def make_bin_matrix(dim, field):
    bin_matrix = np.zeros(dim, dtype=int)
    perm = np.random.permutation(dim[0])
    for i in range(dim[0]):
        bin_matrix[i, perm[i]] = 1
    return field(bin_matrix)

# order by count number of 0, 1, ..., q in a vector (dictionary). A multiset1 < multiset2 if number of 0 > number of 0 of 
# second multiset. If the number of 0 is the same check number of 1 and so on
def order_row3(matrix, error_counter):  
    start = time.perf_counter()#this is for make file csv dati_differences_upd2 and only count/ord
    vector_of_counts = np.zeros((matrix.shape[0], field.order + 1), dtype = int)
    n_colonne = matrix.shape[1]  
    #start = time.perf_counter()
    
    #for row in matrix:
    for i, row in enumerate(matrix):
        #bp.bincout count frequency of element in row
        vector_of_counts[i] = np.bincount(row, minlength = field.order + 1)
    #vector_of_counts = np.array(vector_of_counts)
    if has_duplicate_rows(vector_of_counts):
        error_counter += 1
        #print(vector_of_counts)
        print("Duplicated rows all columns",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    
    sorted_index = np.lexsort([vector_of_counts[:, i] for i in range(n_colonne, -1, -1)])[::-1]
    end = time.perf_counter()
    duration = (end - start) * 1000    
    
    start = time.perf_counter() #this is for make file csv dati_differences_upd2 and only count/ord
    vector_of_counts1 = np.zeros((matrix.shape[0], max_column), dtype = int)
    for i, row in enumerate(matrix):
        #BY REMOVING THIS FILTERING (BELOW) WHICH WOULD ADD OVERHEAD AS I HAVE TO SCROLL THROUGH THE ENTIRE VECTOR REGARDLESS
        #I WILL SAVE IN TERMS OF TIME
        #valid_values = row[row < max_column]
        vector_of_counts1[i] = np.bincount(row, minlength = max_column)[:max_column]
    #print(vector_of_counts1)
    if has_duplicate_rows(vector_of_counts1):
        error_counter += 1
        #print(vector_of_counts1)
        print("Duplicated rows less columns",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    sorted_index1 = np.lexsort([vector_of_counts1[:, i] for i in range(vector_of_counts1.shape[1]-1, -1, -1)])[::-1]
    
    end = time.perf_counter() #these 2 for make file csv dati_differences_upd2
    duration1 = (end - start) * 1000 
    print(np.array_equal(matrix[sorted_index], matrix[sorted_index1]))
    if np.array_equal(matrix[sorted_index], matrix[sorted_index1]):
        print("If positivo")
        return matrix[sorted_index], duration, 0, matrix[sorted_index1], duration1, 0, error_counter
    else:
        print("If negativo")
        error_counter += 1
        print(error_counter)
        return [], duration, 0, [], duration1, 0, error_counter

# order column as always
def sort_by_column(matrix, rows, num_swap_columns):
    #print("Funzione ordina per colonna")
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    initial_position = np.arange(matrix.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe, se c'è conflitto passa a el seconda riga
    print(matrix.shape[0], matrix.shape[1])
    sorting_indices = np.lexsort([matrix[row, :] for row in rows[::-1]])
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, index in enumerate(sorting_indices):
        if index != initial_position[i]:
            num_swap_columns += 1
    
    #riordina la matrice in base agli indici trovati
    sorted_matrix = matrix[:, sorting_indices]
    """for raw in sorted_matrix:
        print(raw)
    print()"""
    return sorted_matrix, num_swap_columns

def algorithm_test3(matrix1, error_counter, k):
    row1_sort, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter = order_row3(matrix1, error_counter)
    #print(row1_sort)
    if len(row1_sort) == 0:
        return False, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter
    matrix1_sorted = sort_by_column(row1_sort, range(k), 0)
    #print(matrix1_sorted)
    return True, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter

In [53]:
#FUNCTION FOR CF 4
def has_duplicate_rows(matrix):
    _, counts = np.unique(matrix, axis=0, return_counts=True)
    return np.any(counts > 1)

def make_bin_matrix(dim, field):
    bin_matrix = np.zeros(dim, dtype=int)
    perm = np.random.permutation(dim[0])
    for i in range(dim[0]):
        bin_matrix[i, perm[i]] = 1
    return field(bin_matrix)

#initial matrix x matrix with s or s^(-1)
def make_matrix_xS(matrix, field_order, k):
    start = time.perf_counter()
    matrix_S = field(np.zeros((k, k), dtype = int))
    for i, row in enumerate(matrix):
        if np.all(row == row[0]):
            if row[0] != 0:
                row[:] = 1
                matrix_S[i,i] = 1
            else:
                matrix_S[i, i] = 1
        else:
            s = np.sum(row)
            #print(s)
            s_prime = np.sum(row ** (field_order-2))
            #print(s_prime)
            if s != 0:
                matrix_S[i,i] = np.reciprocal(s)
            elif s_prime != 0:
                matrix_S[i,i] = s_prime
            else:
                
                print("esco")
                return [], 0
    
    matrix = matrix_S @ matrix
    end = time.perf_counter()
    duration = (end - start) * 1000
    return matrix, duration

# order by count number of 0, 1, ..., q in a vector (dictionary). A multiset1 < multiset2 if number of 0 > number of 0 of 
# second multiset. If the number of 0 is the same check number of 1 and so on
def order_row4(matrix, error_counter, max_column):  
    start = time.perf_counter()#this is for make file csv dati_differences_upd2 and only count/ord
    if len(matrix) == 0:
        #no error_counter += 1 because isn t an error of algoritm
        return [], 0, 0, [], 0, 0, error_counter
    vector_of_counts = np.zeros((matrix.shape[0], field.order + 1), dtype = int)
    n_colonne = matrix.shape[1]  
    
    #for row in matrix:
    for i, row in enumerate(matrix):
        #bp.bincout count frequency of element in row
        vector_of_counts[i] = np.bincount(row, minlength = field.order + 1)

    #vector_of_counts = np.array(vector_of_counts)
    if has_duplicate_rows(vector_of_counts):
        error_counter += 1
        #print(vector_of_counts)
        print("Duplicated rows all columns",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    
    sorted_index = np.lexsort([vector_of_counts[:, i] for i in range(n_colonne, -1, -1)])[::-1]
    end = time.perf_counter() #these 2 for make file csv dati_differences_upd2
    duration = (end - start) * 1000    
    
    start = time.perf_counter() #this is for make file csv dati_differences_upd2 and only count/ord
    if len(matrix) == 0:
        #no error_counter += 1 because isn t an error of algoritm
        return [], 0, 0, [], 0, 0, error_counter
    vector_of_counts1 = np.zeros((matrix.shape[0], max_column), dtype = int)
    for i, row in enumerate(matrix):
        #BY REMOVING THIS FILTERING (BELOW) WHICH WOULD ADD OVERHEAD AS I HAVE TO SCROLL THROUGH THE ENTIRE VECTOR REGARDLESS
        #I WILL SAVE IN TERMS OF TIME
        #valid_values = row[row < max_column]
        vector_of_counts1[i] = np.bincount(row, minlength = max_column)[:max_column]
    #print(vector_of_counts1)
    
    if has_duplicate_rows(vector_of_counts1):
        error_counter += 1
        #print(vector_of_counts1)
        print("Duplicated rows less columns",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    sorted_index1 = np.lexsort([vector_of_counts1[:, i] for i in range(vector_of_counts1.shape[1]-1, -1, -1)])[::-1]
    
    end = time.perf_counter() #these 2 for make file csv dati_differences_upd2
    duration1 = (end - start) * 1000 
    print(np.array_equal(matrix[sorted_index], matrix[sorted_index1]))
    if np.array_equal(matrix[sorted_index], matrix[sorted_index1]):
        print("If positivo")
        return matrix[sorted_index], duration, 0, matrix[sorted_index1], duration1, 0, error_counter
    else:
        print("If negativo")
        error_counter += 1
        print(error_counter)
        return [], duration, 0, [], duration1, 0, error_counter
    
def algorithm_test4(matrix1, error_counter, k):
    row1_sort, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter = order_row4(matrix1, error_counter, max_column)
    #print(row1_sort)
    if len(row1_sort) == 0:
        return False, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter
    matrix1_sorted = sort_by_column(row1_sort, range(k), 0)
    #print(matrix1_sorted)
    return True, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter

In [50]:
#FUNCTION FOR CF 5
def check_non_zero_row(row):
    #return true if all elements != 0
    return np.all(row)

#make scaling matrix for every row with all elements != 0 and A x D added to a vector
def scaled_matrices(matrix, k, n_k):
    start = time.perf_counter()
    scaled_matrices = []
    scaling_matrix = field(np.zeros((k, n_k), dtype=int))
    for i, row in enumerate(matrix):
        if check_non_zero_row(row):
            for j in range(matrix.shape[1]):
                scaling_matrix[j,j] = np.reciprocal(matrix[i,j])
            #print(f"Matrice per scalare\n", scaling_matrix)
            #print("Matrix x scalata")
            #print(matrix @ scaling_matrix)
            scaled_matrices.append(matrix @ scaling_matrix)
    end = time.perf_counter()
    duration = (end - start) * 1000
    return scaled_matrices, duration

#QUINDI CAPISCI COME CONTARE L'ERROR_COUNTER E SE UN ALGORITMO FALLISCE SULLA MATRICE DI UNA RIGA SI DEVE FERMARE TUTTO 
#SU QUELLA MATRICE OPPURE VADO AVANTI ALLA PROSSIMA RIGA. LA SOLUZIONE ME L HA DATA PAOLO: NON ANDIAMO IN ERRORE ANDIAMO 
#SEMPLICEMENTE AVANTI
def apply_cf_4_4(scaled_matrices, error_counter, add_to_iteration, k, max_column):
    start = time.perf_counter()
    tot_duration1_1 = 0
    tot_duration1_2 = 0
    all_matrices = []
    num_swap_col = 0
    if len(scaled_matrices) != 0:
        end = time.perf_counter()
        duration0 = (end - start) * 1000
        for matrix in scaled_matrices:
            start = time.perf_counter()
            add_to_iteration += 1
            end = time.perf_counter()
            duration = (end - start) * 1000
            matrix, dur = make_matrix_xS(matrix, field.order, k)
            row_sort, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter = order_row4(matrix, error_counter, max_column)
            tot_duration1_1 += (duration1_1+duration+duration0)
            tot_duration1_2 += (durations1_2+duration+duration0)
            start = time.perf_counter()
            if not len(row_sort) == 0:
                scaled_matrix_sorted, num_swap_col = sort_by_column(row_sort, range(k), 0)
                #print(scaled_matrix_sorted)
                all_matrices.append(scaled_matrix_sorted)
            #when make an error into cf4 of a matrix for make cf5 i continue and no go in error 
            #else: #there when i go in error not do error_counter += 1 because i do this in other functions
            end = time.perf_counter()
            duration = (end - start) * 1000
            tot_duration1_1 += duration
            tot_duration1_2 += duration
        return True, tot_duration1_1, operations, row1_2_sort, tot_duration1_2, operations1, error_counter, all_matrices, add_to_iteration, duration
    else: 
        return False, 0, 0, 0, 0, 0, error_counter, [], add_to_iteration, 0

def cf5(all_matrices, error_counter, max_column):
    start = time.perf_counter()
    if len(all_matrices) != 0:
        smallest_matrix = np.zeros((len(all_matrices), max_column), dtype = int)
        for i, matrix in enumerate(all_matrices):
            smallest_matrix[i] = np.bincount(matrix[0,:], minlength = max_column)[:max_column]

        if (smallest_matrix.shape[0] > 1): 
            smallest_index = np.lexsort([smallest_matrix[:, i] for i in range(smallest_matrix.shape[1]-1, -1, -1)])[::-1]
            #the row below is very rare with big dimensions. To make an error it is sufficient that the first 2 rows are equal
            if (np.array_equal(smallest_matrix[smallest_index[0]], smallest_matrix[smallest_index[1]])):
                #print("Duplicated rows in frequency first line")
                error_counter += 1
                return [], error_counter, 0 #return empty element for make a check in other funcrtion
            else:
                end = time.perf_counter()
                duration = (end - start) * 1000
                return all_matrices[smallest_index[0]], error_counter, duration
        else:
            end = time.perf_counter()
            duration = (end - start) * 1000
            return all_matrices[0], error_counter, duration
    else:
        return [], error_counter, 0

In [11]:
num_iterations = 500
def montecarlo_simulation(num_iterations):
    vector_operations = []
    vector_operations1 = []
    vector_duration1 = []
    vector_duration1_2 = []
    error_counter_3 = 0
    error_counter_4 = 0
    error_counter_5 = 0
    add_to_iteration = 0
    duration1_accumulator4 = 0
    duration1_2_accumulator4 = 0
    duration1_accumulator5 = 0
    duration1_2_accumulator5 = 0
    duration1_accumulator6 = 0
    duration1_2_accumulator6 = 0
    operations_accumulator = 0
    operations1_accumulator = 0
    csv_file_path = '/home/michele/Desktop/Tirocinio_giusto/Tirocinio/post_uarta_revisione/dati_345.csv'
    file_exists = os.path.exists(csv_file_path)
    #this is for total time
    #with open('dati_tot_time.csv', mode='a', newline='') as file:
    with open(csv_file_path, mode='a', newline='') as file:
        writer_csv = csv.writer(file)
        if not file_exists:
            writer_csv.writerow(['File','Iteration', 'Max_columns', 'Duration_w_all_Columns', 'Duration_w_max_columns', 'Num_op_w_all_columns', 'Num_op_w_max_columns', 'Number of error'])

        for i in range(num_iterations):
            A = field(np.random.randint(0, field.order, size=(k, n_k))) 
            while(has_duplicate_rows(A)):
                A = field(np.random.randint(0, field.order, size=(k, n_k))) 

            #START CF 3
            check, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter_3 = algorithm_test3(A, error_counter_3)
            if check:
                duration1_accumulator4 += duration1_1
                duration1_2_accumulator4 += durations1_2
                operations_accumulator += operations
                operations1_accumulator += operations1
                writer_csv.writerow([4,i, max_column, duration1_1, durations1_2, operations, operations1, error_counter_3])

            
            #START CF 4
            B, dur_xS = make_matrix_xS(A, field.order)
            check, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter_4 = algorithm_test4(B, error_counter_4, k)
            if check:
                duration1_accumulator5 += (duration1_1+dur_xS)
                duration1_2_accumulator5 += (durations1_2+dur_xS)
                operations_accumulator += operations
                operations1_accumulator += operations1
                writer_csv.writerow([5,i, max_column, (duration1_1+dur_xS), (durations1_2+dur_xS), operations, operations1, error_counter_4])

            
            #START CF 5
            matrici_scalate, dura = scaled_matrices(A)
            check, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter_5, all_matrices, add_to_iteration, duration5 = apply_cf_4_4(matrici_scalate, error_counter_5, add_to_iteration)

            if check:
                smallest, error_counter_5, duration = cf5(all_matrices, error_counter_5)
                duration1_accumulator6 += (duration1_1+duration)
                duration1_2_accumulator6 += (durations1_2+duration)
                operations_accumulator += operations
                operations1_accumulator += operations1
                print("Add iteration: ", add_to_iteration)
                if len(smallest) != 0:
                    writer_csv.writerow([6,i, max_column, (dura+duration1_1+duration), (dura+durations1_2+duration), operations, operations1, error_counter_5])
                else:
                    print("Add iteration: ", add_to_iteration)
                    print("Error in cf function", error_counter_5) 
            else:
                print("Add iteration: ", add_to_iteration)
                print("Error in apply _cf_4_4", error_counter_5) #if error_counter not increments error in martrix_S
            
    #return smallest, duration1_accumulator, duration1_2_accumulator, operations_accumulator, operations1_accumulator, (num_iterations-error_counter)/num_iterations

montecarlo_simulation(num_iterations)



NameError: name 'max_column' is not defined

In [7]:
#there i use the NON OPTIMIZED FUNCTION (without count frequency but with sort) so i plot the gain

#function non optimized
def order_row_no_optimized(matrix, error_counter):
    start = time.perf_counter()
    if len(matrix) == 0:
        #no error_counter += 1 because isn t an error of algoritm
        return [], 0, 0, error_counter
    n = matrix.shape[0]
    n_colonne = matrix.shape[1]
    
    # Array indici originali
    indici = np.arange(n)
    indici1 = np.arange(n)

    #start = time.perf_counter()
    ordinato = []
    for riga in matrix:
        ordinato.append(np.sort(riga))
    
    ordinato = np.array(ordinato)
    matrice1 = ordinato
    indici = np.lexsort([matrice1[:, i] for i in range(n_colonne - 1, -1, -1)]) 
    
    #check duplicate rows 
    if has_duplicate_rows(matrice1[indici]):
        error_counter += 1
        tempo = (end - start) * 1000
        print("Algoritmo ha almeno due righe uguali --> fallisco")
        return [], 0, error_counter
    end = time.perf_counter()
    tempo = (end - start) * 1000
    start = time.perf_counter()
    return matrice1[indici], tempo, 0, error_counter

def apply_cf_4_4_non_optimized(scaled_matrices, error_counter, add_to_iteration, k):
    start = time.perf_counter()
    tot_duration1_1 = 0
    tot_duration1_2 = 0
    all_matrices = []
    num_swap_col = 0
    if len(scaled_matrices) != 0:
        end = time.perf_counter()
        duration0 = (end - start) * 1000
        for matrix in scaled_matrices:
            start = time.perf_counter()
            add_to_iteration += 1
            end = time.perf_counter()
            duration = (end - start) * 1000
            matrix, dur = make_matrix_xS(matrix, field.order, k, k)
            row_sort, duration1_1, operations, error_counter = order_row_no_optimized(matrix, error_counter)
            tot_duration1_1 += (duration1_1+duration+duration0)
            start = time.perf_counter()
            if not len(row_sort) == 0:
                scaled_matrix_sorted, num_swap_col = sort_by_column(row_sort, range(k), 0)
                #print(scaled_matrix_sorted)
                all_matrices.append(scaled_matrix_sorted)
            #when make an error into cf4 of a matrix for make cf5 i continue and no go in error 
            #else: #there when i go in error not do error_counter += 1 because i do this in other functions
            end = time.perf_counter()
            duration = (end - start) * 1000
            tot_duration1_1 += duration
            tot_duration1_2 += duration
        return True, tot_duration1_1, operations, error_counter, all_matrices, add_to_iteration, duration
    else: 
        return False, 0, 0, error_counter, [], add_to_iteration, 0

def order_row4_only_all_column(matrix, error_counter):  
    start = time.perf_counter()#this is for make file csv dati_differences_upd2 and only count/ord
    if len(matrix) == 0:
        #no error_counter += 1 because isn t an error of algoritm
        return [], 0, 0, error_counter
    vector_of_counts = np.zeros((matrix.shape[0], field.order + 1), dtype = int)
    n_colonne = matrix.shape[1]  
    
    #for row in matrix:
    for i, row in enumerate(matrix):
        #bp.bincout count frequency of element in row
        vector_of_counts[i] = np.bincount(row, minlength = field.order + 1)

    #vector_of_counts = np.array(vector_of_counts)
    if has_duplicate_rows(vector_of_counts):
        error_counter += 1
        #print(vector_of_counts)
        print("Duplicated rows all columns",error_counter)
        return [], 0, 0, error_counter
    
    sorted_index = np.lexsort([vector_of_counts[:, i] for i in range(n_colonne, -1, -1)])[::-1]
    end = time.perf_counter() #these 2 for make file csv dati_differences_upd2
    duration = (end - start) * 1000    
    return matrix[sorted_index], duration, 0, error_counter

def apply_cf_4_4_only_all_col(scaled_matrices, error_counter, add_to_iteration, k):
    start = time.perf_counter()
    tot_duration1_1 = 0
    tot_duration1_2 = 0
    all_matrices = []
    num_swap_col = 0
    if len(scaled_matrices) != 0:
        end = time.perf_counter()
        duration0 = (end - start) * 1000
        for matrix in scaled_matrices:
            start = time.perf_counter()
            add_to_iteration += 1
            end = time.perf_counter()
            duration = (end - start) * 1000
            matrix, dur = make_matrix_xS(matrix, field.order, k, k)
            row_sort, duration1_1, operations, error_counter = order_row4_only_all_column(matrix, error_counter)
            tot_duration1_1 += (duration1_1+duration+duration0)
            start = time.perf_counter()
            if not len(row_sort) == 0:
                scaled_matrix_sorted, num_swap_col = sort_by_column(row_sort, range(k), 0)
                #print(scaled_matrix_sorted)
                all_matrices.append(scaled_matrix_sorted)
            #when make an error into cf4 of a matrix for make cf5 i continue and no go in error 
            #else: #there when i go in error not do error_counter += 1 because i do this in other functions
            end = time.perf_counter()
            duration = (end - start) * 1000
            tot_duration1_1 += duration
            tot_duration1_2 += duration
        return True, tot_duration1_1, operations, error_counter, all_matrices, add_to_iteration, duration
    else: 
        return False, 0, 0, error_counter, [], add_to_iteration, 0
    
def cf5_all_col(all_matrices, error_counter):
    start = time.perf_counter()
    if len(all_matrices) != 0:
        smallest_matrix = np.zeros((len(all_matrices), field.order + 1), dtype = int)
        for i, matrix in enumerate(all_matrices):
            smallest_matrix[i] = np.bincount(matrix[0,:], minlength = field.order + 1)

        if (smallest_matrix.shape[0] > 1): 
            smallest_index = np.lexsort([smallest_matrix[:, i] for i in range(smallest_matrix.shape[1]-1, -1, -1)])[::-1]
            #the row below is very rare with big dimensions. To make an error it is sufficient that the first 2 rows are equal
            if (np.array_equal(smallest_matrix[smallest_index[0]], smallest_matrix[smallest_index[1]])):
                #print("Duplicated rows in frequency first line")
                error_counter += 1
                return [], error_counter, 0 #return empty element for make a check in other funcrtion
            else:
                end = time.perf_counter()
                duration = (end - start) * 1000
                return all_matrices[smallest_index[0]], error_counter, duration
        else:
            end = time.perf_counter()
            duration = (end - start) * 1000
            return all_matrices[0], error_counter, duration
    else:
        return [], error_counter, 0
    
    
num_iterations = 151
def montecarlo_simulation(num_iterations):
    k = 50 #rows
    n_k = 50 #columns
    error_counter_5 = 0
    error_counter_5_noOp = 0
    add_to_iteration = 0
    add_to_iteration_noOp = 0
    duration1_accumulator6_noOp = 0
    duration1_2_accumulator6_noOp = 0
    duration1_accumulator6 = 0
    duration1_2_accumulator6 = 0
    operations_accumulator = 0
    operations1_accumulator = 0
    csv_file_path = '/home/michele/Desktop/Tirocinio_giusto/Tirocinio/post_uarta_revisione/dati_5_opt&noOpt.csv'
    file_exists = os.path.exists(csv_file_path)
    #this is for total time
    #with open('dati_tot_time.csv', mode='a', newline='') as file:
    with open(csv_file_path, mode='a', newline='') as file:
        writer_csv = csv.writer(file)
        if not file_exists:
            writer_csv.writerow(['File','Iteration', 'Duration_w_all_Columns', 'Num_op_w_all_columns', 'Number of error', 'Round', 'Rows-Column'])

        for i in range(num_iterations):
            print(i)
            k += 1
            n_k += 1
            #print("Field order:", field.order)
            #cosider before 9% of the total columns and after 10% of the total columns (9% of 126 = 11, 10% of 126 = 12)
            print("Righe, Colonne:", k, n_k)
            for i in range(20):
                A = field(np.random.randint(0, field.order, size=(k, n_k))) 
                while(has_duplicate_rows(A)):
                    A = field(np.random.randint(0, field.order, size=(k, n_k)))
                print(A.shape[0], A.shape[1])
 
                    
                #START CF 5
                matrici_scalate, dura = scaled_matrices(A, k, n_k)
                check, duration1_1, operations, error_counter_5, all_matrices, add_to_iteration, duration5 = apply_cf_4_4_only_all_col(matrici_scalate, error_counter_5, add_to_iteration, k)
                #non optimized
                check_noOp, duration1_1_noOp, operations, error_counter_5_noOp, all_matrices, add_to_iteration_noOp, duration5_noOp = apply_cf_4_4_non_optimized(matrici_scalate, error_counter_5, add_to_iteration_noOp, k)
                if check and check_noOp:
                    smallest, error_counter_5, duration = cf5_all_col(all_matrices, error_counter_5,)
                    smallest_noOp, error_counter_5_noOp, duration_noOp = cf5_all_col(all_matrices, error_counter_5_noOp)
                    duration1_accumulator6 += (duration1_1+duration)
                    duration1_accumulator6_noOp += (duration1_1_noOp + duration_noOp)
                    operations_accumulator += operations
                    print("Add iteration: ", add_to_iteration)
                    print("Add iteration no op: ", add_to_iteration_noOp)
                    if len(smallest) != 0:
                        writer_csv.writerow([6, i, (dura+duration1_1+duration), operations, error_counter_5, i, k])
                    else:
                        print("Add iteration: ", add_to_iteration)
                        print("Error in cf function", error_counter_5) 
                    if len(smallest_noOp) != 0:
                        writer_csv.writerow([7, i, (dura+duration1_1_noOp+duration_noOp), operations, error_counter_5_noOp, i, k])
                    else:
                        print("Add iteration no op: ", add_to_iteration_noOp)
                        print("Error in cf function", error_counter_5_noOp) 
                else:
                    print("Add iteration: ", add_to_iteration)
                    print("Error in apply _cf_4_4", error_counter_5) #if error_counter not increments error in martrix_S
                    print("Add iteration no op: ", add_to_iteration_noOp)
                    print("Error in apply _cf_4_4 no op", error_counter_5_noOp) #if error_counter not increments error in martrix_S
                
    #return smallest, duration1_accumulator, duration1_2_accumulator, operations_accumulator, operations1_accumulator, (num_iterations-error_counter)/num_iterations

montecarlo_simulation(num_iterations)
    

0
Righe, Colonne: 51 51
51 51


Add iteration:  35
Add iteration no op:  35
51 51
esco
esco
esco
esco
Add iteration:  70
Add iteration no op:  70
51 51
Add iteration:  102
Add iteration no op:  102
51 51
Add iteration:  140
Add iteration no op:  140
51 51
Add iteration:  177
Add iteration no op:  177
51 51
esco
esco
esco
esco
Add iteration:  212
Add iteration no op:  212
51 51
Add iteration:  249
Add iteration no op:  249
51 51
Add iteration:  281
Add iteration no op:  281
51 51
Add iteration:  320
Add iteration no op:  320
51 51
Add iteration:  358
Add iteration no op:  358
51 51
Add iteration:  393
Add iteration no op:  393
51 51
Add iteration:  428
Add iteration no op:  428
51 51
Add iteration:  459
Add iteration no op:  459
51 51
Add iteration:  494
Add iteration no op:  494
51 51
Add iteration:  532
Add iteration no op:  532
51 51
Add iteration:  572
Add iteration no op:  572
51 51
Add iteration:  606
Add iteration no op:  606
51 51
Add iteration:  635
Add iteration no op:  635
51 51
Add iteration:  663
Add iter

IndexError: index 128 is out of bounds for axis 1 with size 128

In [56]:
num_iterations = 100
def montecarlo_simulation(num_iterations):
    error_counter_3 = 0
    error_counter_4 = 0
    error_counter_5 = 0
    add_to_iteration = 0
    duration1_accumulator4 = 0
    duration1_2_accumulator4 = 0
    duration1_accumulator5 = 0
    duration1_2_accumulator5 = 0
    duration1_accumulator6 = 0
    duration1_2_accumulator6 = 0
    operations_accumulator = 0
    operations1_accumulator = 0
    csv_file_path = '/home/michele/Desktop/Tirocinio_giusto/Tirocinio/post_uarta_revisione/dati_345_n_fixed.csv'
    file_exists = os.path.exists(csv_file_path)
    #this is for total time
    #with open('dati_tot_time.csv', mode='a', newline='') as file:
    with open(csv_file_path, mode='a', newline='') as file:
        writer_csv = csv.writer(file)
        if not file_exists:
            writer_csv.writerow(['File','Iteration', 'n', 'Max_columns', 'Duration_w_all_Columns', 'Duration_w_max_columns', 'Num_op_w_all_columns', 'Num_op_w_max_columns', 'Number of error'])
        #these are the 3 value of n
        for j in [126, 190, 252]:
            k = int(j/2)
            n_k = k
            for i in range(num_iterations):
                A = field(np.random.randint(0, field.order, size=(k, n_k))) 
                while(has_duplicate_rows(A)):
                    A = field(np.random.randint(0, field.order, size=(k, n_k))) 

                #START CF 3
                check, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter_3 = algorithm_test3(A, error_counter_3, k)
                if check:
                    duration1_accumulator4 += duration1_1
                    duration1_2_accumulator4 += durations1_2
                    operations_accumulator += operations
                    operations1_accumulator += operations1
                    writer_csv.writerow([4, i, j, max_column, duration1_1, durations1_2, operations, operations1, error_counter_3])

                
                #START CF 4
                B, dur_xS = make_matrix_xS(A, field.order, k)
                check, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter_4 = algorithm_test4(B, error_counter_4, k)
                if check:
                    duration1_accumulator5 += (duration1_1+dur_xS)
                    duration1_2_accumulator5 += (durations1_2+dur_xS)
                    operations_accumulator += operations
                    operations1_accumulator += operations1
                    writer_csv.writerow([5, i, j, max_column, (duration1_1+dur_xS), (durations1_2+dur_xS), operations, operations1, error_counter_4])

                
                #START CF 5
                matrici_scalate, dura = scaled_matrices(A, k, n_k)
                check, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter_5, all_matrices, add_to_iteration, duration5 = apply_cf_4_4(matrici_scalate, error_counter_5, add_to_iteration, k, max_column)

                if check:
                    smallest, error_counter_5, duration = cf5(all_matrices, error_counter_5, max_column)
                    duration1_accumulator6 += (duration1_1+duration)
                    duration1_2_accumulator6 += (durations1_2+duration)
                    operations_accumulator += operations
                    operations1_accumulator += operations1
                    print("Add iteration: ", add_to_iteration)
                    if len(smallest) != 0:
                        writer_csv.writerow([6, i, j, max_column, (dura+duration1_1+duration), (dura+durations1_2+duration), operations, operations1, error_counter_5])
                    else:
                        print("Add iteration: ", add_to_iteration)
                        print("Error in cf function", error_counter_5) 
                else:
                    print("Add iteration: ", add_to_iteration)
                    print("Error in apply _cf_4_4", error_counter_5) #if error_counter not increments error in martrix_S
                
    #return smallest, duration1_accumulator, duration1_2_accumulator, operations_accumulator, operations1_accumulator, (num_iterations-error_counter)/num_iterations

montecarlo_simulation(num_iterations)



True
If positivo
63 63
True
If positivo
63 63
True
If positivo
63 63
True
If positivo
63 63
True
If positivo
63 63
True
If positivo
63 63
Duplicated rows less columns 1
Duplicated rows less columns 2
True
If positivo
63 63
True
If positivo
63 63
Duplicated rows less columns 3
True
If positivo
63 63
True
If positivo
63 63
True
If positivo
63 63
True
If positivo
63 63
True
If positivo
63 63
True
If positivo
63 63
Duplicated rows less columns 4
True
If positivo
63 63
True
If positivo
63 63
Duplicated rows less columns 5
True
If positivo
63 63
Duplicated rows less columns 6
True
If positivo
63 63
True
If positivo
63 63
Duplicated rows less columns 7
Duplicated rows less columns 8
True
If positivo
63 63
True
If positivo
63 63
True
If positivo
63 63
True
If positivo
63 63
Duplicated rows less columns 9
True
If positivo
63 63
Duplicated rows less columns 10
Duplicated rows less columns 11
True
If positivo
63 63
Duplicated rows less columns 12
True
If positivo
63 63
True
If positivo
63 63
Add 